In [1]:

#using Pkg
#Pkg.add("CSV")
#Pkg.add("DataFrames")
#Pkg.add("JuMP")
#Pkg.add("Gurobi")



In [1]:
using CSV, DataFrames, JuMP, Gurobi

In [2]:
facilities = CSV.read("facilities.csv", DataFrame)
@show nrow(facilities)
facilities
supply = facilities[:,3]
supply
sup_cor = facilities[:,1:2]
sup_cor

nrow(facilities) = 100


Row,x,y,supply
,Float64,Float64,Float64
1,0.0285239,1.71448,86.7759
2,0.508327,2.76107,140.987
3,3.60514,4.41322,213.104
4,8.43778,1.25384,102.001
5,3.76959,5.45553,409.883
6,4.89495,8.96613,576.407
7,1.80321,6.49412,205.929
8,3.25198,0.451999,325.911
9,4.39766,4.72416,317.815


In [10]:
customers = CSV.read("customers.csv", DataFrame)
@show nrow(customers)
customers
demand = customers[:,3]
demand
dem_cor = customers[:,1:2]
dem_cor

nrow(customers) = 500


Row,x,y
,Float64,Float64
1,7.37441,2.06318
2,1.00208,7.29743
3,3.61792,1.25644
4,2.27311,2.9264
5,0.765783,9.5186
6,2.03323,9.76306
7,9.92024,1.5894
8,4.3794,9.87982
9,5.69553,8.78813


In [24]:
cost = [sqrt((sup_cor[i,1]-dem_cor[j,1])^2+(sup_cor[i,2]-dem_cor[j,2])^2) for j in 1:500 for i in 1:100]
cost = reshape(cost,(100,500))


100×500 Matrix{Float64}:
 7.35416   5.6672   3.6185    2.55087  …  2.95463  8.94706   2.59445
 6.90146   4.56316  3.45448   1.77251     2.72912  8.51112   1.48807
 4.44186   3.88518  3.15681   1.99623     2.87529  5.93534   3.0164
 1.33633   9.582    4.81987   6.38754     5.46799  0.826434  8.39598
 4.95003   3.32442  4.20184   2.9387      3.929    6.31277   3.3985
 7.33475   4.23545  7.81475   6.58425  …  7.60444  8.17397   6.38567
 7.1184    1.13452  5.54315   3.59853     5.02762  8.52558   2.55238
 4.4261    7.2057   0.883758  2.66099     1.18579  5.89817   4.63337
 3.99273   4.26049  3.55431   2.7831      3.42441  5.38672   3.83416
 5.64988   6.65181  1.81356   2.25399     1.41513  7.18103   3.77149
 ⋮                                     ⋱                     
 6.15368   6.35871  7.87109   7.29337     7.89249  6.55265   7.78828
 2.4539   10.6991   5.7492    7.43477     6.42863  1.35288   9.47326
 4.48844   4.73145  5.31408   4.65886     5.27655  5.46794   5.37049
 1.90734   8.59858

In [40]:
model = Model(Gurobi.Optimizer)
@variable(model, X[1:100, 1:500]);
@objective(model, Min, sum(sum(cost[i,j]*X[i,j] for j in 1:500) for i in 1:100));
@constraint(model,a_constraint[i in 1:100 ,j in 1:500],X[i,j] >= 0 );
@constraint(model,d_constraint[j in 1:500],sum(X[i,j] for i in 1:100) >= demand[j] );
@constraint(model,s_constraint[i in 1:100],sum(X[i,j] for j in 1:500) <= supply[i] );
optimize!(model)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-01
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i7-6567U CPU @ 3.30GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 50600 rows, 50000 columns and 150000 nonzeros
Model fingerprint: 0x212be045
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-02, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-01, 1e+03]
Presolve removed 50000 rows and 0 columns
Presolve time: 0.08s
Presolved: 600 rows, 50000 columns, 100000 nonzeros

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 5.000e+04
 Factor NZ  : 6.521e+04 (roughly 20 MB of memory)
 Factor Ops : 7.808e+06 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal   

In [27]:
objective_value(model)

23274.33843049142

In [39]:
cnt = 0
for j in 1:500
    # find the nearest
    cj = cost[:,j]
    mVal,i = findmin(cj)
    cost_tmp = abs(value(X[i,j]) - demand[j])
    if cost_tmp < 1e-6
        cnt+=1
    end
end
print(cnt)

220

In [12]:
println(stderr, "hello!")

hello!


In [13]:
model2 = Model(Gurobi.Optimizer)
@variable(model2, X2[1:1000, 1:20]);
@objective(model2, Min, sum(sum(Menergy[i,j]*X2[i,j] for j in 1:20) for i in 1:1000));
@constraint(model2,a_constraint[i in 1:1000 ,j in 1:20],X2[i,j] >= 0 );
@constraint(model2,b_constraint[i in 1:1000],sum(X2[i,j] for j in 1:20) == 1 );
#@constraint(model2,c_constraint[j in 1:20],sum(X2[i,j] for i in 1:1000) <= Mcapacity[j] );

Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-01


In [14]:
optimize!(model2)

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i7-6567U CPU @ 3.30GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 21000 rows, 20000 columns and 40000 nonzeros
Model fingerprint: 0xb147ac93
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e+00, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 21000 rows and 20000 columns
Presolve time: 0.02s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.0883161e+04   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.04 seconds (0.01 work units)
Optimal objective  4.088316111e+04

User-callback calls 38, time in user-callback 0.00 sec


In [20]:
cnt = 0

for i in 1:100
    for j in 1:20
        cnt += abs(value(X[i,j]) - value(X2[i,j]))
    end
end
println(cnt/2)

39.0
